The International Monetary Fund [estimates](https://www.imf.org/en/Publications/WP/Issues/2019/05/02/Global-Fossil-Fuel-Subsidies-Remain-Large-An-Update-Based-on-Country-Level-Estimates-46509) that the United States subsidized $649 billion of fossil fuels in 2015.
Meanwhile, the Environmental and Energy Study Institute (EASE) [estimates](https://www.eesi.org/papers/view/fact-sheet-fossil-fuel-subsidies-a-closer-look-at-tax-breaks-and-societal-costs) that only about $20 billion of that is in the form of direct subsidies.
The remainder is the _implicit_ subsidy of misaligning the price of fossil fuels with their societal harms like air pollution and climate change.
That is, the vast majority of fossil fuel subsidies is the _absence of carbon pricing_.

All developed countries except the [US and Australia](https://citizensclimatelobby.org/laser-talks/carbon-prices-around-world/) cut down on fossil fuel subsidies by pricing carbon emissions.
Experts including [scientists](https://academic.oup.com/bioscience/advance-article/doi/10.1093/biosci/biab079/6325731) and the [vast majority](https://www.igmchicago.org/surveys/carbon-taxes-ii/) of [leading economists](http://econstatement.org) have called on the US to follow suit.
To offset higher prices consumers would face as a result of carbon pricing (or other climate measures), many of these experts suggest rebating the revenue to all Americans as a dividend.
[Two thirds](https://clcouncil.org/morning-consult-poll.pdf) of American voters support this policy, and as of this writing, 80 House Democrats have cosponsored the [Energy Innovation and Carbon Dividend Act](https://energyinnovationact.org/) (EICDA) which would make it a reality in the US.

Beyond the climate impacts of carbon dividend---policies like the EICDA are [projected](https://www.nature.com/articles/s41558-020-0880-3) to meet the IPCC's goal of net zero emissions by 2050 and [avert 295,000 deaths](https://www.nature.com/articles/nclimate2935) from air pollution in the first decade---many economists have [documented the](https://www.energypolicy.columbia.edu/sites/default/files/pictures/CGEP_Distributional_Implications_CarbonTax.pdf) [progressive](https://taxfoundation.org/carbon-tax/) [distributional](https://www.rff.org/publications/data-tools/carbon-pricing-calculator/) [impacts](https://www.peoplespolicyproject.org/wp-content/uploads/2018/09/CarbonTax.pdf) of carbon dividends.
As one example, University of Pennsylvania researcher [Kevin Ummel found](https://citizensclimatelobby.org/household-impact-study/) found that that EICDA would benefit 96% of households in the bottom consumption quintile and 12% in the top quintile, after considering effects on prices, capital income, and the dividend.

Using Kevin Ummel's data[^ummel] and the 2018 American Community Survey,[^acsspm] I extend this research to estimate the poverty impacts of a carbon dividend[^cpsp] across a range of prices.
I find that carbon dividends reduce poverty roughly linearly with respect to the price, for example by 15% at $100 per ton, though the effects are larger for both child poverty and deep poverty, and especially deep child poverty.

[^ummel]: I'm grateful to Kevin Ummel for sharing and explaining his data and for his permission to use it in this post.

[^acsspm]: I use the [American Community Survey Supplemental Poverty Measure research file](https://www.census.gov/data/datasets/time-series/demo/supplemental-poverty-measure/acs-research-files.html), which estimates the poverty measure inclusive of taxes, transfers, and housing costs, historically only available in the [Current Population Survey](https://www.census.gov/library/publications/2020/demo/p60-272.html).

[^cpsp]: The only other study I'm aware of that estimates the impact of a carbon dividend on poverty is from the [Columbia Center on Poverty and Social Policy](https://www.povertycenter.columbia.edu/news-internal/carbontax). They found that a carbon dividend of $42 per ton would reduce poverty by 1.1% and child poverty by 4.6%, where I found effects of 4% and 8%, respectively, for a $40 carbon dividend. Our analyses differed in terms of carbon footprint estimation in the Consumer Expenditure Survey (CPSP only considered direct energy consumption, where Ummel's data considered all consumption categories), poverty dataset (CPSP used the Current Population Survey where I used the American Community Survey), and other factors (e.g., CPSP imputed receipt of underreported benefits with the TRIM3 program, where I did not).


# How big would the dividend be?

Carbon pricing proposals vary in their initial price and the rate at which they increase over time, though they do all increase in some way.
The EICDA starts at $15 per metric ton in 2021 and rises by $10 per year, while the Senate's [Save Our Future Act](https://www.whitehouse.senate.gov/news/release/whitehouse-and-schatz-introduce-save-our-future-act-to-charge-big-polluters-for-emissions-redirect-trillions-to-american-families-and-communities-harmed-by-pollution) starts at $54 per ton in 2023 and rises by an inflation-adjusted 6% per year (about $4 in the first year).
Since American households currently emit about 5 billion metric tons of carbon per year, amounting to about 15 tons per person, each $10 carbon price would translate to a monthly dividend of 15 tons * $10 / 12 months = **$13**.

In [1]:
import pandas as pd
import plotly.express as px
from ubicenter import format_fig

# Remove when exporting and creating in vscode.
# import plotly.io as pio

# pio.renderers.default = "notebook"

# Use same sources from script.
# Adjust population and CO2 to match administrative sources for 2021.
# Source: https://www.macrotrends.net/countries/USA/united-states/population
TRUE_POP = 332_915_073
# https://data.census.gov/cedsci/table?q=us%20population%20by%20age&tid=ACSST1Y2019.S0101
TRUE_KID_SHARE = 72_967_785 / 328_239_523
# Source: https://www.rff.org/publications/data-tools/carbon-pricing-calculator
TRUE_CO2 = 5.05e9
total_kids = TRUE_POP * TRUE_KID_SHARE
total_adults = TRUE_POP - total_kids
total_shares = total_adults + total_kids * 0.5
co2_per_share = TRUE_CO2 / total_shares

# Define colors from material design.
BLUE = "#2196F3"
DARK_BLUE = "#0D47A1"  # 900
LIGHT_BLUE = "#90CAF9"
GRAY = "#BDBDBD"

LABELS = dict(
    price="Carbon price",
    poverty_chg="Change to poverty rate",
    deep_poverty_chg="Change to deep poverty rate",
    age_group="Age group",
    state="State",
    poverty_str="",
    deep_poverty_str="",
    base_poverty="Current poverty rate",
    base_deep_poverty="Current deep poverty rate",
    poverty="Reform poverty rate",
    deep_poverty="Reform deep poverty rate",
    metric="Metric",
    impact="Poverty impact",
    monthly_dividend="Monthly dividend",
)

# Chart of dividend by carbon price.
dividend_by_carbon_price = pd.DataFrame(dict(price=range(0, 105, 5)))
dividend_by_carbon_price["monthly_dividend"] = (
    co2_per_share * dividend_by_carbon_price.price / 12
).round(2)
fig = px.line(
    dividend_by_carbon_price,
    "price",
    "monthly_dividend",
    title="Monthly carbon dividend per adult by per-ton carbon price",
    labels=LABELS,
    color_discrete_sequence=[DARK_BLUE],
    hover_data=dict(monthly_dividend=":$.0f"),
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="$")
format_fig(fig)

# How would the carbon dividend affect overall poverty?

Poverty and deep poverty (the population share with income below half their poverty threshold) fall roughly linearly with respect to the carbon price: each $10 carbon price would reduce poverty by 1% and deep poverty by 1.2%. A carbon dividend of $100 per ton would reduce poverty by 10% and deep poverty by 12%.

In [2]:
carbon_price = pd.read_csv("../data/carbon_price_half_kids.csv")
# For now stick to prices up to $100.
carbon_price = carbon_price[carbon_price.price <= 100]

carbon_price_long_us = carbon_price[carbon_price.state_abbr == "US"].melt(
    ["age_group", "price"],
    ["poverty_chg", "deep_poverty_chg"],
    var_name="metric",
    value_name="impact",
)
carbon_price_long_us.metric = carbon_price_long_us.metric.map(
    dict(poverty_chg="Poverty", deep_poverty_chg="Deep poverty")
)

# Overall poverty and deep poverty.
fig = px.line(
    carbon_price_long_us[carbon_price_long_us.age_group == "All"],
    "price",
    "impact",
    color="metric",
    title="Carbon dividend poverty impact by carbon price",
    labels=LABELS,
    color_discrete_map={"Poverty": BLUE, "Deep poverty": DARK_BLUE},
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="%", legend_title=None)
format_fig(fig)

# How would the carbon dividend affect poverty by age?

Child and adult poverty also fall linearly with the carbon price, but child poverty falls about three times as much. A $100 carbon price would reduce child poverty by 19% and deep child poverty by 27%.

In [3]:
fig = px.bar(
    carbon_price_long_us[carbon_price_long_us.price > 5],
    "age_group",
    "impact",
    color="metric",
    animation_frame="price",
    title="Carbon dividend poverty impact by age group",
    labels=LABELS,
    color_discrete_map={"Poverty": BLUE, "Deep poverty": DARK_BLUE},
)
fig.update_layout(
    yaxis_tickformat="%",
    barmode="group",
    yaxis_range=[carbon_price_long_us.impact.min() * 1.1, 0],
    legend_title=None,
)
format_fig(fig)

# How would the carbon dividend affect poverty by state?

Poverty impacts also vary geographically, largely in line with states' per-capita carbon emissions, which in turn correlate to [cold weather](https://doi.org/10.1016/S0928-7655(00)00027-0) and [low population density](https://www.sciencedirect.com/science/article/abs/pii/S0301421516300167?via%3Dihub). For example, consider the two extremes of how carbon dividends lower poverty:

* California ranks [#10 on average temperature](http://www.usa.com/rank/us--average-temperature--state-rank.htm), [#3 on population density](https://skyscraperpage.com/forum/showthread.php?t=211827), [#49 on per-capita carbon emissions](https://www.eia.gov/environment/emissions/state/analysis/), and #1 in terms of poverty reduction from a carbon dividend. A \$100 per ton carbon dividend would lower poverty in California by 14%, and lower deep child poverty by 30%.
* North Dakota ranks #49 on average temperature, #32 on population density, #2 on per-capita carbon emissions, and #50 in terms of poverty reduction from a carbon dividend. In fact, North Dakota is the only state for which a carbon dividend would increase poverty, though only by 3% at $100 per ton; child poverty would still fall, and its poverty rate would remain one of the lowest in the country.

In [4]:
map_data_all = carbon_price[
    (carbon_price.state_abbr != "US")
    & (carbon_price.age_group == "All")
    & (carbon_price.price > 5)
].copy()
map_data_child = (
    carbon_price[
        (carbon_price.state_abbr != "US")
        & (carbon_price.age_group == "Child")
        & (carbon_price.price > 5)
    ]
    .copy()
    .rename(
        columns=dict(
            base_poverty="base_child_poverty",
            poverty="child_poverty",
            poverty_chg="child_poverty_chg",
            base_deep_poverty="base_deep_child_poverty",
            deep_poverty="deep_child_poverty",
            deep_poverty_chg="deep_child_poverty_chg",
        )
    )
)
map_data = map_data_all.merge(
    map_data_child, on=["price", "state_abbr", "state_name"]
)


def pov_str(name, base, new):
    chg = (new / base) - 1
    if chg > 0:
        fall_rise = f"rises {chg:.0%}"
    else:
        fall_rise = f"falls {-chg:.0%}"
    return f"{name} {fall_rise} from {base:.1%} to {new:.1%}"


map_data["poverty_string"] = map_data.apply(
    lambda x: pov_str("Poverty", x.base_poverty, x.poverty), axis=1
)
map_data["deep_poverty_string"] = map_data.apply(
    lambda x: pov_str("Deep poverty", x.base_deep_poverty, x.deep_poverty),
    axis=1,
)
map_data["child_poverty_string"] = map_data.apply(
    lambda x: pov_str("Child poverty", x.base_child_poverty, x.child_poverty),
    axis=1,
)
map_data["deep_child_poverty_string"] = map_data.apply(
    lambda x: pov_str(
        "Deep child poverty", x.base_deep_child_poverty, x.deep_child_poverty
    ),
    axis=1,
)
map_data["poverty_fall"] = -map_data.poverty_chg
fig = px.choropleth(
    map_data,
    locations="state_abbr",
    locationmode="USA-states",
    color="poverty_fall",
    scope="usa",
    animation_frame="price",
    title="Poverty impact of a carbon dividend by US state and price",
    labels=LABELS,
    custom_data=["state_name", "poverty_string", "deep_poverty_string", "child_poverty_string", "deep_child_poverty_string"],
    color_continuous_scale=px.colors.sequential.Blues,
    range_color=[0, map_data.poverty_fall.max()],
)
CUSTOM_HOVERTEMPLATE = (
    "<b>%{customdata[0]}</b><br>%{customdata[1]}<br>%{customdata[2]}<br>%{customdata[3]}<br>%{customdata[4]}"
)
fig.update_traces(hovertemplate=CUSTOM_HOVERTEMPLATE)
# See https://github.com/plotly/plotly.py/issues/2722.
for frame in fig.frames:
    frame.data[0].hovertemplate = CUSTOM_HOVERTEMPLATE
fig.update_layout(coloraxis_showscale=False)
format_fig(fig)

Politicians from [Bernie Sanders](https://grist.org/climate-energy/sanders-and-boxer-introduce-fee-and-dividend-climate-bill-greens-tickled-pink/) to [Pete Buttigieg](https://thehill.com/policy/energy-environment/444330-buttigieg-climate-plan-includes-a-carbon-tax) to [Mitt Romney](https://twitter.com/citizensclimate/status/1369059303995437058) have endorsed carbon dividends, primarily framing the dividend as a way to offset potential regressive effects of the carbon price.
However, the dividend more than neutralizes the carbon price: it makes a significant reduction to poverty in America. A $20 carbon dividend would cut poverty more than a fully-phased-in $15 minimum wage would,[^mw] while a $100 carbon dividend (as the US would reach within a decade under proposed legislation) would approach the impact of the [expanded Child Tax Credit](http://ubicenter.org/advance-ctc).[^ctc]

My estimates may even be conservative. For example, I assume that 100% of the carbon price is passed on to consumers, but [one study finds](https://cowles.yale.edu/sites/default/files/files/pub/d20/d2038-r.pdf) that only 70% is passed on. In his analysis of the EICDA, Ummel applies pass-through rates of 70%, 85%, and 100%, with the remainder borne by people in proportion to their investable assets; he finds that lower pass-through assumptions yield more progressive distributional impacts.

Carbon dividends would also create infrastructure for further cash assistance.
When the US government had to keep Americans afloat with relief checks in the Covid-19 pandemic, they relied on the IRS, and largely failed to reach nonfilers.
Had even a small universal payment program existed at the time, they could have simply increased the amount instead.
The carbon dividend is a simple policy that can help the US progress on a slew of objectives: climate change, air pollution, financial resilience, and, yes, even poverty.

[^mw]: The Congressional Budget Office [estimated](https://www.cbo.gov/publication/55681) that the Raise the Wage Act of 2021 would reduce the number of people in poverty by 0.7 million by 2028, or about 1.8% of the roughly 40 million people in poverty. A $20 carbon dividend would lower poverty by 2%. CBO applied the Official Poverty Measure rather than Supplemental Poverty Measure.

[^ctc]: Carbon dividends may also have greater success fulfilling their theoretical antipoverty potential, since they are universal and thus can be administered outside the tax system. The Child Tax Credit is expected to reach only 90% of eligible children, and the remaining 10% are disproportionately poor children whose parents don't earn enough to have to file taxes.


# Appendix

In [5]:
carbon_price = pd.read_csv("../data/carbon_price.csv")

carbon_price_long_us = carbon_price[carbon_price.state_abbr == "US"].melt(
    ["age_group", "price"],
    ["poverty_chg", "deep_poverty_chg"],
    var_name="metric",
    value_name="impact",
)
carbon_price_long_us.metric = carbon_price_long_us.metric.map(
    dict(poverty_chg="Poverty", deep_poverty_chg="Deep poverty")
)

# Overall poverty and deep poverty.
fig = px.line(
    carbon_price_long_us[carbon_price_long_us.age_group == "All"],
    "price",
    "impact",
    color="metric",
    title="Carbon dividend poverty impact by carbon price",
    labels=LABELS,
    color_discrete_map={"Poverty": BLUE, "Deep poverty": DARK_BLUE},
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="%", legend_title=None)
format_fig(fig)